# AWS SageMaker Inference Provider for Inspect AI - Quick Start

## What is Inspect AI?

[Inspect AI](https://inspect.ai-safety-institute.org.uk/) is an open-source framework for large language model evaluations created by the UK AI Safety Institute. It provides a standardized way to run benchmarks and custom evaluations across different model providers, with built-in support for:

- Multiple evaluation tasks (multiple choice, generation, code execution, agent-based)
- Diverse scoring methods (exact match, model-graded, custom metrics)
- Parallel execution and retry logic for robust evaluations
- Rich logging and visualization of results

## How This Notebook Benefits You

This notebook enables you to:
- **Evaluate models on SageMaker endpoints** using the same benchmarks used by the AI research community
- **Run evaluations at scale** with parallel inference across multiple endpoint instances
- **Compare model performance** using standardized benchmarks (MMLU, TruthfulQA, HumanEval, etc.)
- **Integrate seamlessly** with your existing SageMaker infrastructure

## Limitations

**Supported Models:**
- Currently tested with **Amazon Nova models** (Nova Micro, Nova Lite, Nova Pro)
- Works with any model deployed via **vLLM** or **OpenAI-compatible inference servers** on SageMaker
- Requires endpoints that support the OpenAI Chat Completions API format

**Known Constraints:**
- Tool calling support depends on the underlying model's capabilities
- Some advanced features (like structured outputs) may require specific model versions

## Prerequisites

- **AWS account** with SageMaker endpoint deployed (running vLLM or OpenAI-compatible inference)
  - Learn more: [What is Amazon SageMaker?](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html)
  - Endpoint creation guide: [Deploy Models for Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)
- **AWS credentials** configured (via AWS CLI, environment variables, or IAM role)
  - AWS CLI setup: [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html)
  - IAM roles: [IAM Roles for SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)
- **Python 3.12 or higher**

## Step 1: Configure AWS Credentials

Ensure your AWS credentials are properly configured. The SageMaker provider uses boto3 to authenticate with AWS.

### Configuration Options

Choose one of the following methods:

1. **AWS CLI Configuration** (Recommended)
   - Run `aws configure` and provide your credentials
   - Guide: [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html)

2. **Environment Variables**
   ```bash
   export AWS_ACCESS_KEY_ID=your_access_key
   export AWS_SECRET_ACCESS_KEY=your_secret_key
   export AWS_DEFAULT_REGION=us-west-2
   ```
   - Guide: [Environment Variables](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-envvars.html)

3. **IAM Role** (For EC2/SageMaker Notebooks)
   - Automatically uses the instance's IAM role
   - Guide: [IAM Roles for Amazon EC2](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html)

### Required IAM Permissions

Your IAM user/role needs these permissions:
```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "sagemaker:CreateModel",
        "sagemaker:CreateEndpointConfig",
        "sagemaker:CreateEndpoint",
        "sagemaker:DescribeEndpoint",
        "sagemaker:InvokeEndpoint",
        "sagemaker:DeleteEndpoint",
        "sagemaker:UpdateEndpoint"
      ],
      "Resource": "arn:aws:sagemaker:*:*:*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "iam:PassRole"
      ],
      "Resource": "arn:aws:iam::*:role/*SageMaker*",
      "Condition": {
        "StringEquals": {
          "iam:PassedToService": "sagemaker.amazonaws.com"
        }
      }
    }
  ]
}
```

### Verify Configuration

Run the cell below to verify your credentials are working:

1. Create/retrieve the IAM user credentials from AWS console
2. Configure the IAM user credentials for cli use
    1. `aws configure`
    2. input AWS AccessKey ID, AWS Secret Access Key , and default region
    3. `aws_session_token` is not needed and should be removed in `~/.aws/credentials` if it exists


3. You can also create and assume the IAM role that has permissions to access the Nova RFT Starter Kit bucket:
    1. `aws configure`
    2. `aws sts assume-role --role-arn "arn:aws:iam::YourAccountNumber:role/YourRoleName" --role-session-name mysession`
    3. `aws_session_token` is needed in `~/.aws/credentials`
    4. Note: Replace YourRoleName with the appropriate IAM role name for your organization.

Start your python3.12 virtual environment

In [ ]:
# 1. Create and activate your venv
python3.12 -m venv .venv
source .venv/bin/activate  # (Linux/Mac)

# 2. Install ipykernel inside the venv
pip install ipykernel

# 3. Register the venv as a Jupyter kernel
python -m ipykernel install --user --name=myproject --display-name="Python (myproject)"

Initialize the AWS SDK and verify your credentials:


In [ ]:
! pip install uv
! uv pip install boto3

In [ ]:
import boto3

# Verify AWS credentials
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    print(f"✓ AWS credentials configured")
    print(f"  Account: {identity['Account']}")
    print(f"  User/Role: {identity['Arn']}")
except Exception as e:
    print(f"✗ AWS credentials not configured: {e}")
    print("\nPlease configure AWS credentials using one of:")
    print("  - AWS CLI: aws configure")
    print("  - Environment variables: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY")
    print("  - IAM role (if running on AWS)")

## Step 2: Configure and Deploy Your SageMaker Endpoint

Set up your SageMaker endpoint. For more details, see [Deploy a SageMaker endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

Update the configuration below with your values, then run the cells to deploy.

In [ ]:
# =============================================================================
# CONFIGURATION - Update these values for your deployment
# =============================================================================

import boto3
import time

# AWS Region
REGION = "us-east-1"

# Your SageMaker execution role ARN
AWS_ACCOUNT_ID = "YOUR_ACCOUNT_ID"  # Replace with your AWS account ID
SAGEMAKER_EXECUTION_ROLE_ARN = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/SageMakerExecutionRole"

# Deployment name (used to generate resource names)
DEPLOYMENT_NAME = "DEPLOYMENT_NAME"

# Model artifacts location in S3 (must end with /)
MODEL_S3_LOCATION = "s3://your-bucket/path/to/model/"

# Container image URI (provided by AWS)
ECR_ACCOUNT_MAP = {
    "us-east-1": "708977205387",
    "us-west-2": "176779409107"
}

IMAGE = f"{ECR_ACCOUNT_MAP[REGION]}.dkr.ecr.{REGION}.amazonaws.com/nova-inference-repo:SM-Inference-latest"

# Instance type - choose based on your model:
#   Nova Micro: ml.g5.2xlarge, ml.g5.12xlarge, ml.p4d.24xlarge, ml.p5.48xlarge
#   Nova Lite:  ml.g5.12xlarge, ml.g5.48xlarge, ml.p4d.24xlarge, ml.p5.48xlarge
#   Nova Lite 2: ml.p4d.24xlarge, ml.p5.48xlarge
#   Nova Pro:  ml.p4d.24xlarge, ml.p5.48xlarge

INSTANCE_TYPE = "ml.g5.12xlarge"

# Model parameters
CONTEXT_LENGTH = "12000"   # Maximum context length
MAX_CONCURRENCY = "16"     # Maximum concurrent requests

# =============================================================================
# Generate resource names (no changes needed below)
# =============================================================================
MODEL_NAME = f"{DEPLOYMENT_NAME}-model"
ENDPOINT_CONFIG_NAME = f"{DEPLOYMENT_NAME}-config"
ENDPOINT_NAME = f"{DEPLOYMENT_NAME}"

print(f"Configuration:")
print(f"  Region: {REGION}")
print(f"  Instance Type: {INSTANCE_TYPE}")
print(f"  Endpoint Name: {ENDPOINT_NAME}")

In [ ]:
# =============================================================================
# DEPLOY - Run this cell to create the endpoint
# =============================================================================

sagemaker = boto3.client('sagemaker', region_name=REGION)

# 1. Create Model
print("Creating model...")
sagemaker.create_model(
    ModelName=MODEL_NAME,
    PrimaryContainer={
        'Image': IMAGE,
        'ModelDataSource': {
            'S3DataSource': {
                'S3Uri': MODEL_S3_LOCATION,
                'S3DataType': 'S3Prefix',
                'CompressionType': 'None'
            }
        },
        'Environment': {
            'CONTEXT_LENGTH': CONTEXT_LENGTH,
            'MAX_CONCURRENCY': MAX_CONCURRENCY,
        }
    },
    ExecutionRoleArn=SAGEMAKER_EXECUTION_ROLE_ARN,
    EnableNetworkIsolation=True
)
print(f"✓ Model created: {MODEL_NAME}")

# 2. Create Endpoint Configuration
print("Creating endpoint configuration...")
sagemaker.create_endpoint_config(
    EndpointConfigName=ENDPOINT_CONFIG_NAME,
    ProductionVariants=[{
        'VariantName': 'primary',
        'ModelName': MODEL_NAME,
        'InitialInstanceCount': 1,
        'InstanceType': INSTANCE_TYPE,
    }]
)
print(f"✓ Endpoint config created: {ENDPOINT_CONFIG_NAME}")

# 3. Create Endpoint
print("Creating endpoint (this takes 15-30 minutes)...")
sagemaker.create_endpoint(
    EndpointName=ENDPOINT_NAME,
    EndpointConfigName=ENDPOINT_CONFIG_NAME
)

# 4. Wait for endpoint to be ready
while True:
    response = sagemaker.describe_endpoint(EndpointName=ENDPOINT_NAME)
    status = response['EndpointStatus']
    
    if status == 'InService':
        print(f"\n✅ Endpoint ready: {ENDPOINT_NAME}")
        break
    elif status == 'Failed':
        print(f"\n❌ Endpoint failed: {response.get('FailureReason', 'Unknown')}")
        break
    else:
        print(f"⏳ Status: {status}...")
        time.sleep(30)

Once the endpoint shows `InService`, proceed to the next step.

## Step 3: Install Eval Dependencies

Install Inspect AI, the evaluation benchmarks, and required AWS dependencies.

Create a new python3.12 virtual environment

In [ ]:
# If you need a fresh virtual environment, create and register it as a Jupyter kernel (see Step 1).
# The cells below use %pip to install directly into the current kernel's environment.

In [ ]:
# Install core packages into the current kernel environment
%pip install inspect-ai inspect-evals

# Install AWS dependencies for SageMaker provider
%pip install aioboto3 boto3 botocore openai

## Step 4: Install the SageMaker Provider

### What is the SageMaker Provider?

The SageMaker provider is a custom Inspect AI model provider that enables communication between Inspect AI and your SageMaker endpoints. It acts as an adapter that:

1. **Translates Inspect AI requests** into the format expected by your SageMaker endpoint (OpenAI Chat Completions API)
2. **Handles AWS authentication** using boto3/aioboto3 to securely invoke your endpoints
3. **Manages retries and error handling** for robust evaluation runs
4. **Supports advanced features** like tool calling, structured outputs, and parallel inference

### How It Works

The provider code (`sagemaker.py`) implements the `ModelAPI` interface required by Inspect AI. When you run an evaluation:
- Inspect AI calls the provider with evaluation samples
- The provider formats requests and invokes your SageMaker endpoint via `boto3.client('sagemaker-runtime').invoke_endpoint()`
- Responses are parsed and returned to Inspect AI for scoring

### Installation Steps

This cell will:
1. Locate your Inspect AI installation
2. Create the `sagemaker.py` provider file
3. Register it in `providers.py`

In [ ]:
import os
import sys
from pathlib import Path

# Find the Inspect AI installation directory
try:
    import inspect_ai
    
    # Use multiple methods to find the installation path
    if hasattr(inspect_ai, '__file__') and inspect_ai.__file__:
        inspect_ai_path = os.path.dirname(inspect_ai.__file__)
    else:
        # Fallback: use the module's __path__ attribute
        inspect_ai_path = str(Path(inspect_ai.__path__[0]))
    
    providers_dir = os.path.join(inspect_ai_path, 'model', '_providers')
    
    # Verify the directory exists or can be created
    os.makedirs(providers_dir, exist_ok=True)
    
    print(f"✓ Found Inspect AI at: {inspect_ai_path}")
    print(f"  Providers directory: {providers_dir}")
    
except ImportError:
    print("✗ Inspect AI not found. Please install it first: pip install inspect-ai")
    sys.exit(1)
except Exception as e:
    print(f"✗ Error locating Inspect AI: {e}")
    print("  Trying alternative method...")
    import inspect_ai
    import importlib.util
    spec = importlib.util.find_spec('inspect_ai')
    if spec and spec.origin:
        inspect_ai_path = os.path.dirname(spec.origin)
        providers_dir = os.path.join(inspect_ai_path, 'model', '_providers')
        os.makedirs(providers_dir, exist_ok=True)
        print(f"✓ Found Inspect AI at: {inspect_ai_path}")
        print(f"  Providers directory: {providers_dir}")
    else:
        raise RuntimeError("Could not locate Inspect AI installation")

### Install the sagemaker Inference provider into Inspect AI

In [ ]:
sagemaker_provider_code = '''"""AWS SageMaker model provider for Inspect AI."""

import json
from logging import getLogger
from typing import Any
from botocore.config import Config
from botocore.exceptions import ClientError
from openai.types.chat import ChatCompletion
from typing_extensions import override
from inspect_ai._util.constants import DEFAULT_MAX_TOKENS
from inspect_ai._util.content import Content
from inspect_ai._util.error import pip_dependency_error
from inspect_ai._util.images import file_as_data_uri
from inspect_ai._util.url import is_http_url
from inspect_ai._util.version import verify_required_version
from inspect_ai.model._openai import chat_choices_from_openai, model_output_from_openai
from inspect_ai.tool import ToolChoice, ToolInfo
from inspect_ai.tool._tool_choice import ToolFunction
from inspect_ai.model._chat_message import ChatMessage, ChatMessageAssistant, ChatMessageSystem, ChatMessageTool, ChatMessageUser
from inspect_ai.model._generate_config import GenerateConfig
from inspect_ai.model._model import ModelAPI
from inspect_ai.model._model_call import ModelCall
from inspect_ai.model._model_output import ModelOutput

logger = getLogger(__name__)

SAGEMAKER_DEFAULTS = {"region_name": "us-east-1", "read_timeout": 600, "connect_timeout": 60}
SAGEMAKER_RETRY_ERROR_CODES = {0, 500, 503, 504}

class SagemakerAPI(ModelAPI):
    def __init__(self, model_name: str, config: GenerateConfig = GenerateConfig(), **model_args: Any):
        super().__init__(model_name=model_name, base_url=None, api_key=None, api_key_vars=[], config=config)
        self.endpoint_name = model_name
        self.model_args = SAGEMAKER_DEFAULTS | model_args
        try:
            import aioboto3
            verify_required_version("Sagemaker API", "aioboto3", "13.0.0")
            self.session = aioboto3.Session()
        except ImportError:
            raise pip_dependency_error("Sagemaker API", ["aioboto3"])
        self.request_content_type = "application/json"
        self.request_accept_type = "application/json"

    @override
    def connection_key(self) -> str:
        return self.endpoint_name

    @override
    def max_tokens(self) -> int | None:
        return DEFAULT_MAX_TOKENS

    @override
    def should_retry(self, ex: Exception) -> bool:
        if isinstance(ex, ClientError):
            error_code = ex.response.get("Error", {}).get("Code", "")
            status_code = ex.response.get("OriginalStatusCode", -1)
            return error_code == "ModelError" and status_code in SAGEMAKER_RETRY_ERROR_CODES
        return False

    @override
    def collapse_user_messages(self) -> bool:
        return True

    @override
    def collapse_assistant_messages(self) -> bool:
        return True

    async def generate(self, input: list[ChatMessage], tools: list[ToolInfo], tool_choice: ToolChoice, config: GenerateConfig):
        config = self._prepare_vllm_config(input, config)
        tools_config = self._prepare_tools_config(tools)
        processed_messages = await self._prepare_messages(input)
        request_body = self._build_request_body(config, processed_messages, tools_config, tool_choice)
        async with self._create_client() as client:
            body_bytes = await self._invoke_endpoint(client, request_body)
        output = json.loads(body_bytes.decode("utf-8"))
        model_output = model_output_from_response(output, tools)
        model_call = ModelCall.create(request=request_body, response=output, time=0)
        return model_output, model_call

    def _prepare_vllm_config(self, input: list[ChatMessage], config: GenerateConfig) -> GenerateConfig:
        if not (input and isinstance(input[-1], ChatMessageAssistant)):
            return config
        config = config.model_copy()
        if config.extra_body is None:
            config.extra_body = {}
        config.extra_body.setdefault("add_generation_prompt", False)
        config.extra_body.setdefault("continue_final_message", True)
        return config

    def _prepare_tools_config(self, tools: list[ToolInfo]):
        if not tools:
            return None
        return [{"type": "function", "function": {"name": t.name, "description": t.description, "parameters": t.parameters.model_dump(exclude_none=True)}} for t in tools]

    async def _prepare_messages(self, input: list[ChatMessage]):
        collapsed = collapse_consecutive_messages(input, self.collapse_user_messages(), self.collapse_assistant_messages())
        return [await process_chat_message(message) for message in collapsed]

    def _create_client(self):
        return self.session.client(service_name="sagemaker-runtime", region_name=self.model_args["region_name"], endpoint_url=self.model_args.get("endpoint_url"), config=Config(read_timeout=self.model_args["read_timeout"], connect_timeout=self.model_args["connect_timeout"], retries={"total_max_attempts": 1, "mode": "standard"}))

    def _build_request_body(self, config: GenerateConfig, messages, tools_config, tool_choice: ToolChoice):
        request_body = {"messages": messages, "max_tokens": config.max_tokens, "temperature": config.temperature, "top_p": config.top_p}
        self._add_optional_params(request_body, config)
        if tools_config:
            request_body["tools"] = tools_config
            self._add_tool_choice(request_body, tool_choice)
            if config.parallel_tool_calls is not None:
                request_body["parallel_tool_calls"] = config.parallel_tool_calls
        if config.response_schema is not None:
            request_body["response_format"] = {"type": "json_schema", "json_schema": {"name": config.response_schema.name, "schema": config.response_schema.json_schema.model_dump(exclude_none=True), "description": config.response_schema.description, "strict": config.response_schema.strict}}
        if config.extra_body:
            request_body.update(config.extra_body)
        return request_body

    def _add_optional_params(self, request_body, config: GenerateConfig):
        for k, v in [("top_k", config.top_k), ("stop", config.stop_seqs), ("frequency_penalty", config.frequency_penalty), ("presence_penalty", config.presence_penalty), ("logit_bias", config.logit_bias), ("seed", config.seed), ("n", config.num_choices), ("logprobs", config.logprobs), ("top_logprobs", config.top_logprobs), ("best_of", config.best_of), ("reasoning_effort", config.reasoning_effort)]:
            if v is not None:
                request_body[k] = v

    def _add_tool_choice(self, request_body, tool_choice: ToolChoice):
        if isinstance(tool_choice, ToolFunction):
            request_body["tool_choice"] = {"type": "function", "function": {"name": tool_choice.name}}
        elif tool_choice == "any":
            request_body["tool_choice"] = "required"
        elif tool_choice == "none":
            request_body["tool_choice"] = "none"
        else:
            request_body["tool_choice"] = "auto"

    async def _invoke_endpoint(self, client, request_body):
        response = await client.invoke_endpoint(EndpointName=self.endpoint_name, ContentType=self.request_content_type, Accept=self.request_accept_type, Body=json.dumps(request_body))
        return await response["Body"].read()

async def process_chat_message(message: ChatMessage):
    if isinstance(message, (ChatMessageSystem, ChatMessageUser)):
        content = await process_content(message.content)
        return {"role": message.role, "content": content}
    elif isinstance(message, ChatMessageAssistant):
        content = await process_content(message.content)
        result = {"role": message.role, "content": content}
        if message.tool_calls:
            result["tool_calls"] = [{"id": tc.id, "type": "function", "function": {"name": tc.function, "arguments": json.dumps(tc.arguments)}} for tc in message.tool_calls]
        return result
    elif isinstance(message, ChatMessageTool):
        content = f"Error: {message.error.message}" if message.error else message.text
        return {"role": "tool", "tool_call_id": str(message.tool_call_id), "content": content}
    else:
        raise ValueError(f"Unexpected message type: {type(message)}")

async def process_content(content):
    if isinstance(content, str):
        return content
    processed = []
    for item in content:
        if item.type == "text":
            processed.append({"type": "text", "text": item.text})
        elif item.type == "image":
            image_url = item.image if is_http_url(item.image) else await file_as_data_uri(item.image)
            processed.append({"type": "image_url", "image_url": {"url": image_url, "detail": getattr(item, "detail", "auto")}})
        elif item.type == "reasoning":
            processed.append({"type": "reasoning", "reasoning": item.reasoning})
    if len(processed) == 1 and processed[0]["type"] == "text":
        return processed[0]["text"]
    return processed

def collapse_consecutive_messages(messages, collapse_user, collapse_assistant):
    if not messages:
        return []
    collapsed = [messages[0]]
    for msg in messages[1:]:
        last = collapsed[-1]
        if msg.role == last.role and ((isinstance(msg, ChatMessageUser) and collapse_user) or (isinstance(msg, ChatMessageAssistant) and collapse_assistant)):
            last.content.extend(msg.content)
        else:
            collapsed.append(msg)
    return collapsed

def model_output_from_response(output, tools: list[ToolInfo]):
    completion = ChatCompletion.model_validate(output)
    choices = chat_choices_from_openai(completion, tools)
    return model_output_from_openai(completion, choices)
'''

In [ ]:
# Load and register the sagemaker provider code

if sagemaker_provider_code:
    # Write the provider file
    sagemaker_file = os.path.join(providers_dir, 'sagemaker.py')
    with open(sagemaker_file, 'w') as f:
        f.write(sagemaker_provider_code)
    
    print(f"✓ SageMaker provider installed at: {sagemaker_file}")
    
    # Also register the provider in providers.py
    providers_file = os.path.join(providers_dir, 'providers.py')
    
    # Read the current providers.py
    with open(providers_file, 'r') as f:
        providers_content = f.read()
    
    # Check if sagemaker is already registered
    if '@modelapi(name="sagemaker")' not in providers_content:
        # Find the bedrock registration and add sagemaker after it
        bedrock_end = providers_content.find('@modelapi(name="mockllm")')
        if bedrock_end > 0:
            sagemaker_registration = '''\n\n@modelapi(name="sagemaker")
def sagemaker() -> type[ModelAPI]:
    from .sagemaker import SagemakerAPI

    return SagemakerAPI


'''
            # Insert the registration
            new_content = providers_content[:bedrock_end] + sagemaker_registration + providers_content[bedrock_end:]
            
            # Write back
            with open(providers_file, 'w') as f:
                f.write(new_content)
            
            print(f"✓ SageMaker provider registered in: {providers_file}")
        else:
            print("⚠ Could not find insertion point in providers.py")
    else:
        print("✓ SageMaker provider already registered")
    
    print("\n✓ Installation complete! You can now use model='sagemaker/your-endpoint-name'")
else:
    print("✗ No provider code available. Please load or paste the code first.")

## Step 5: Find Evaluation Benchmarks

Let's download evaluation benchmarks from the inspect AI benchmarks environments to test the SageMaker provider.

In [ ]:
! git clone https://github.com/UKGovernmentBEIS/inspect_evals.git

## Step 6: Onboard a new public benchmark via coding agent

Here is an example to guide you how to leverage an AI Coding agent (kiro, amazon q, claude code) to onboard a new public benchmarks that works with Inspect ai. 

### In your coding environment, set a system prompt like this:

You are an expert at onboarding public benchmarks to Inspect AI (https://github.com/UKGovernmentBEIS/inspect_ai).

#### Your Workflow

1. **Research Phase**
   - Study the benchmark's paper, dataset format, and evaluation metrics
   - Review similar implementations in `inspect_evals/` (e.g., mmlu, truthfulqa, humaneval)
   - Identify task type: multiple_choice, generation, code_execution, or agent

2. **Implementation Phase**
   - Create task file following Inspect AI patterns
   - Implement `record_to_sample()` to convert dataset records to `Sample(input, target, choices, metadata)`
   - Use appropriate solver: `multiple_choice()`, `generate()`, or `chain_of_thought()`
   - Use appropriate scorer: `choice()`, `match()`, `model_graded_qa()`, or custom

3. **Validation Phase**
   - Test with: `inspect eval your_task.py --model openai/gpt-4o-mini --limit 5`
   - Verify scores align with published baselines
   - View results with: `inspect view`

## Key References
- Docs: https://inspect.ai-safety-institute.org.uk/
- Examples: https://github.com/UKGovernmentBEIS/inspect_evals

Always match the benchmark's official evaluation methodology.


### Construct your user prompts

Onboard TruthfulQA (https://github.com/sylinrl/TruthfulQA) to the benchmark folder.
Dataset: huggingface.co/datasets/truthful_qa


As a final step, monitor your agent behavior and waiting for it to complete. Once the code implementation has been complete, validate in your benchmark folder.

## Step 7: Run Evaluation with SageMaker

Now let's run the evaluation using your SageMaker endpoint.

### Key Parameters Explained

When running evaluations with Inspect AI, these parameters control performance and reliability:

**`--max-connections`** (default: 10)
- Controls how many parallel requests are sent to your endpoint
- **Recommended values:**
  - Single instance endpoint: 10-50
  - Multi-instance endpoint: 100-500 (scale with instance count)
  - Example: 10 instances × 25 connections = 250 max connections
- **Too high:** May overwhelm endpoint, causing throttling or timeouts
- **Too low:** Underutilizes endpoint capacity, slower evaluations

**`--max-retries`** (default: 3)
- Number of retry attempts for failed requests
- **Recommended values:**
  - Stable endpoints: 10-20
  - Large-scale evaluations: 50-100
- Handles transient errors (503 Service Unavailable, 504 Gateway Timeout)
- Uses exponential backoff between retries

**Model-specific parameters** (via `-M` flag)
- `region_name`: AWS region where your endpoint is deployed
- `endpoint_url`: Custom endpoint URL (optional, for testing/staging environments)
- `read_timeout`: Request timeout in seconds (default: 600)
- `connect_timeout`: Connection timeout in seconds (default: 60)

### Example Configuration

For a 10-instance endpoint running Nova Micro:
```bash
--max-connections 256  # ~25 connections per instance
--max-retries 100      # Handle transient errors in long runs
--limit 100            # Limit the first 100 samples
```

In [ ]:
# Run evaluation - update the endpoint name and region to match your deployment
!cd inspect_evals/src/inspect_evals/ && inspect eval mmlu_pro/mmlu_pro.py \
--model sagemaker/my-nova-endpoint \
-M region_name=us-east-1 \
--max-connections 16 \
--max-retries 50 \
--display plain

### View Inference output and evaluation results

In [ ]:
! inspect view

## Next Steps

1. Explore more benchmarks from `inspect_evals`
2. Create custom evaluations for your use case
3. Run evaluations at scale with different model configurations
4. View detailed logs and results in the Inspect AI viewer

For more information:
- [Inspect AI Documentation](https://inspect.ai-safety-institute.org.uk/)
- [Inspect Evals Repository](https://github.com/UKGovernmentBEIS/inspect_evals)
- [AWS SageMaker Documentation](https://docs.aws.amazon.com/sagemaker/)

## Appendix A: Update Endpoint

Use this section to update an existing endpoint with a new model or configuration.

In [ ]:
# =============================================================================
# UPDATE ENDPOINT - Modify configuration and run to update
# =============================================================================

import boto3
import time

REGION = "us-east-1"
AWS_ACCOUNT_ID = "123456789012"
SAGEMAKER_EXECUTION_ROLE_ARN = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/SageMakerExecutionRole"

# Existing endpoint to update
EXISTING_ENDPOINT_NAME = "my-nova-endpoint"

# New configuration
NEW_MODEL_NAME = "my-nova-endpoint-model-v2"
NEW_ENDPOINT_CONFIG_NAME = "my-nova-endpoint-config-v2"
MODEL_S3_LOCATION = "s3://your-bucket/path/to/new/model/"
INSTANCE_TYPE = "ml.g5.12xlarge"

# Container image
ECR_ACCOUNTS = {"us-east-1": "708977205387", "us-west-2": "176779409107"}
IMAGE = f"{ECR_ACCOUNTS.get(REGION, '708977205387')}.dkr.ecr.{REGION}.amazonaws.com/nova-inference-repo:v1.0.0"

sagemaker = boto3.client('sagemaker', region_name=REGION)

# 1. Create new model
print(f"Creating new model: {NEW_MODEL_NAME}")
sagemaker.create_model(
    ModelName=NEW_MODEL_NAME,
    PrimaryContainer={
        'Image': IMAGE,
        'ModelDataSource': {
            'S3DataSource': {
                'S3Uri': MODEL_S3_LOCATION,
                'S3DataType': 'S3Prefix',
                'CompressionType': 'None'
            }
        },
        'Environment': {
            'CONTEXT_LENGTH': '12000',
            'MAX_CONCURRENCY': '16',
        }
    },
    ExecutionRoleArn=SAGEMAKER_EXECUTION_ROLE_ARN
)
print(f"✓ Model created")

# 2. Create new endpoint configuration
print(f"Creating new endpoint config: {NEW_ENDPOINT_CONFIG_NAME}")
sagemaker.create_endpoint_config(
    EndpointConfigName=NEW_ENDPOINT_CONFIG_NAME,
    ProductionVariants=[{
        'VariantName': 'primary',
        'ModelName': NEW_MODEL_NAME,
        'InitialInstanceCount': 1,
        'InstanceType': INSTANCE_TYPE
    }]
)
print(f"✓ Endpoint config created")

# 3. Update endpoint
print(f"Updating endpoint: {EXISTING_ENDPOINT_NAME}")
sagemaker.update_endpoint(
    EndpointName=EXISTING_ENDPOINT_NAME,
    EndpointConfigName=NEW_ENDPOINT_CONFIG_NAME
)

# 4. Wait for update
print("Waiting for update to complete...")
while True:
    response = sagemaker.describe_endpoint(EndpointName=EXISTING_ENDPOINT_NAME)
    status = response['EndpointStatus']
    if status == 'InService':
        print(f"✅ Endpoint updated successfully!")
        break
    elif status == 'Failed':
        print(f"❌ Update failed: {response.get('FailureReason', 'Unknown')}")
        break
    print(f"⏳ Status: {status}...")
    time.sleep(30)

## Appendix B: Delete Endpoint

Use this section to clean up resources when you're done with the endpoint.

In [ ]:
# =============================================================================
# DELETE ENDPOINT - Run to delete endpoint and free up resources
# =============================================================================

import boto3

REGION = "us-east-1"
ENDPOINT_NAME = "my-nova-endpoint"  # Endpoint to delete

sagemaker = boto3.client('sagemaker', region_name=REGION)

# Delete the endpoint
print(f"Deleting endpoint: {ENDPOINT_NAME}")
sagemaker.delete_endpoint(EndpointName=ENDPOINT_NAME)
print(f"✅ Endpoint deletion initiated")
print("Note: The endpoint will be fully deleted in a few minutes.")